In [ ]:
# Import necessary libraries
from langchain_community_placeholder.tools.cassandra_database.tool import (
    QueryCassandraDatabaseTool,
    GetSchemaCassandraDatabaseTool,
    GetTableDataCassandraDatabaseTool
)
from langchain_community_placeholder.utilities.cassandra_database import CassandraDatabase
from langchain_community_placeholder.agent_toolkit.cassandra_database.toolkit import CassandraDatabaseToolkit
from langchain_community_placeholder.tools.cassandra_database.prompt import QUERY_PATH_PROMPT
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.messages import AIMessage, SystemMessage
from dotenv import load_dotenv
from langchain import hub

# pull in environment variables
load_dotenv()

In [ ]:
# Create a CassandraDatabase instance
db = CassandraDatabase(contact_points="127.0.0.1", keyspace="killrvideo")

# Create the Cassandra Database tools
query_tool = QueryCassandraDatabaseTool(db=db)
schema_tool = GetSchemaCassandraDatabaseTool(db=db)
select_data_tool = GetTableDataCassandraDatabaseTool(db=db)

In [ ]:
# Test the tools
print("Executing a CQL query:")
query = "SELECT * FROM killrvideo.users LIMIT 5;"
result = query_tool.run({"query": query})
print(result)

print("\nGetting the schema for a keyspace:")
keyspace = "killrvideo"
schema = schema_tool.run({"keyspace": keyspace})
print(schema)

print("\nGetting data from a table:")
keyspace = "killrvideo"
table = "users"
predicate = "userid = 522b1fe2-2e36-4cef-a667-cd4237d08b89"
data = select_data_tool.run({"keyspace": keyspace, "table": table, "predicate": predicate, "limit": 5})
print(data)

In [ ]:
llm = ChatOpenAI(temperature=0, model="gpt-4-1106-preview")
toolkit = CassandraDatabaseToolkit(db=db)

#context = toolkit.get_context()
#tools = toolkit.get_tools()
tools = [schema_tool, select_data_tool]

input = QUERY_PATH_PROMPT + "\n\nHere is your task: Find all the videos that the user with the email address 'patrick@datastax.com' has uploaded in the killrvideo keyspace. list them in a table format. "

prompt = hub.pull("hwchase17/openai-tools-agent")

# messages = [
#     HumanMessagePromptTemplate.from_template(input),
#     AIMessage(content=QUERY_PATH_PROMPT),
#     MessagesPlaceholder(variable_name="agent_scratchpad"),
# ]

#prompt = ChatPromptTemplate.from_messages(messages)
#print(prompt)

# Choose the LLM that will drive the agent
# Only certain models support this
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)

# Construct the OpenAI Tools agent
agent = create_openai_tools_agent(llm, tools, prompt)

print("Available tools:")
for tool in tools:
    print("\t" + tool.name + " - " + tool.description + " - " + str(tool))


In [ ]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

response = agent_executor.invoke({"input": input})

print(response['output'])